In [1]:
# Install dependencies (only needs to be ran once)

!pip install -q transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.5 MB/s eta 0:00:00


In [ ]:
# Import Model (only needs to be ran once)

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import getpass

USER = getpass.getuser()

# List of different LLMs that we may want to use or tryout. You can lookup each one on https://huggingface.co/
llm_ids = [
    "deepseek-ai/DeepSeek-Prover-V2-7B",
    "Goedel-LM/Goedel-Prover-V2-8B",
    "Goedel-LM/Goedel-Prover-SFT",
    "Goedel-LM/Goedel-Prover-DPO",
    "ByteDance-Seed/BFS-Prover",
    "internlm/internlm2-step-prover",
    "internlm/internlm2_5-step-prover",
    "AI-MO/Kimina-Prover-RL-1.7B"
]

# Select Deepseek LLM
llm_id = llm_ids[0]

tokenizer = AutoTokenizer.from_pretrained(llm_id, cache_dir=f"/work/classtmp/{USER}/models")
model = AutoModelForCausalLM.from_pretrained(llm_id, device_map="auto", cache_dir=f"/work/classtmp/{USER}/models", torch_dtype=torch.bfloat16, trust_remote_code=True)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
`rope_scaling`'s factor field must be a float >= 1, got 16
`rope_scaling`'s beta_fast field must be a float, got 32
`rope_scaling`'s beta_slow field must be a float, got 1


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.22G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.60G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Inference

# This causes the outputs to be deterministic (comment out if this is not desired)
torch.manual_seed(30)

# This prompt was taken from the Deepseek page so it will likely not work with theother LLMs
formal_statement = """
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
""".strip()

prompt = """
Complete the following Lean 4 code:

```lean4
{}
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.
""".strip()

chat = [
  {"role": "user", "content": prompt.format(formal_statement)},
]

inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt", padding=True, truncation=True).to(model.device)

import time
start = time.time()
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs)[0])
print(time.time() - start)

<>:15: SyntaxWarning: invalid escape sequence '\%'
<>:15: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipython-input-917020580.py:15: SyntaxWarning: invalid escape sequence '\%'
  /-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.<｜Assistant｜>### Detailed Proof and Analysis

First, we need to understand the problem:

**Problem:** What is the positive difference between \(120\%\) of \(30\) and \(130\%\) of \(20\)? Show that it is \(10\).

**Solution:**

1. **Calculate \(120\%\) of \(30\):**
   \[
   \frac{120}{100} \times 30 = 1.2 \times 30 = 36
   \]

2. **Calculate \(130